# PineconeEmbeddings

这将帮助您使用LangChain开始使用PineconeEmbeddings的[嵌入模型](/docs/concepts/embedding_models)。有关`PineconeEmbeddings`功能和配置选项的详细文档，请参阅[API参考](https://api.js.langchain.com/classes/_langchain_pinecone.PineconeEmbeddings.html)。

## 概览
### 集成详情

| 类 | 包 | 本地 | [Python支持](https://python.langchain.com/docs/integrations/text_embedding/pinecone/) | 包下载量 | 包最新版本 |
| :--- | :--- | :---: | :---: |  :---: | :---: |
| [PineconeEmbeddings](https://api.js.langchain.com/classes/_langchain_pinecone.PineconeEmbeddings.html) | [@langchain/pinecone](https://api.js.langchain.com/classes/_langchain_pinecone.PineconeEmbeddings.html) | ❌ | ✅ | ![NPM - 下载量](https://img.shields.io/npm/dm/@langchain/pinecone?style=flat-square&label=%20&) | ![NPM - 版本](https://img.shields.io/npm/v/@langchain/pinecone?style=flat-square&label=%20&) |

## 准备

要访问Pinecone的嵌入模型，您需要创建一个Pinecone账户，获取API密钥，并安装`@langchain/pinecone`集成包。

### 凭证

注册一个[Pinecone](https://www.pinecone.io/)账户，获取您的API密钥，并将其设置为名为`PINECONE_API_KEY`的环境变量：

```typescript
process.env.PINECONE_API_KEY = "your-pinecone-api-key";
```

如果您希望自动追踪模型调用，也可以取消下面的注释以设置您的[LangSmith](https://docs.smith.langchain.com/) API密钥：

```bash
# export LANGSMITH_TRACING="true"
# export LANGSMITH_API_KEY="your-api-key"
```

### 安装

LangChain PineconeEmbeddings集成位于`@langchain/pinecone`包中：

```{=mdx}
import IntegrationInstallTooltip from "@mdx_components/integration_install_tooltip.mdx";
import Npm2Yarn from "@theme/Npm2Yarn";

<IntegrationInstallTooltip></IntegrationInstallTooltip>

<Npm2Yarn>
  @langchain/pinecone @langchain/core @pinecone-database/pinecone@5
</Npm2Yarn>
```

## 实例化

现在我们可以实例化我们的模型对象并生成聊天补全：

In [1]:
import { PineconeEmbeddings } from "@langchain/pinecone";

const embeddings = new PineconeEmbeddings({
  model: "multilingual-e5-large",
});

## 索引与检索

嵌入模型通常用于检索增强生成（RAG）流程中，既用于数据索引，也用于后续的数据检索。有关更详细的说明，请参阅我们位于[使用外部知识教程](/docs/tutorials/#working-with-external-knowledge)下的RAG教程。

下面展示如何使用我们上面初始化的`embeddings`对象进行数据索引和检索。在此示例中，我们将使用演示用的[`MemoryVectorStore`](/docs/integrations/vectorstores/memory)对一个示例文档进行索引和检索。

In [2]:
// Create a vector store with a sample text
import { MemoryVectorStore } from "langchain/vectorstores/memory";

const text = "LangChain is the framework for building context-aware reasoning applications";

const vectorstore = await MemoryVectorStore.fromDocuments(
  [{ pageContent: text, metadata: {} }],
  embeddings,
);

// Use the vector store as a retriever that returns a single document
const retriever = vectorstore.asRetriever(1);

// Retrieve the most similar text
const retrievedDocuments = await retriever.invoke("What is LangChain?");

retrievedDocuments[0].pageContent;

LangChain is the framework for building context-aware reasoning applications


## 直接使用

在底层，向量存储和检索器的实现分别通过调用 `embeddings.embedDocument(...)` 和 `embeddings.embedQuery(...)` 来为其在 `fromDocuments` 和检索器的 `invoke` 操作中使用的文本创建嵌入。

您可以直接调用这些方法，以用于您自己的用例。

### 嵌入单个文本

您可以使用 `embedQuery` 对查询进行嵌入以进行搜索。这将生成针对该查询的向量表示：

In [3]:
const singleVector = await embeddings.embedQuery(text);

console.log(singleVector.slice(0, 100));

[
         0.0191650390625,  0.004924774169921875,     -0.015838623046875,
          -0.04248046875,     0.040191650390625,      -0.02679443359375,
     -0.0240936279296875,     0.058624267578125,      0.027069091796875,
        -0.0435791015625,      0.01934814453125,      0.040191650390625,
     -0.0194244384765625,   0.01386260986328125,    -0.0216827392578125,
    -0.01073455810546875,   -0.0166168212890625,    0.01073455810546875,
        -0.0228271484375,       0.0062255859375,      0.035064697265625,
     -0.0114593505859375,   -0.0257110595703125,    -0.0285797119140625,
        0.01190185546875,    -0.022186279296875,   -0.01500701904296875,
       -0.03240966796875, 0.0019063949584960938,     -0.039337158203125,
     -0.0047454833984375,              -0.03125,       -0.0123291015625,
    -0.00899505615234375,        -0.02880859375,      0.014678955078125,
         0.0452880859375,      0.05035400390625,     -0.053436279296875,
      0.0265960693359375,   -0.0206756591796875, 

### 嵌入多个文本

您可以使用 `embedDocuments` 对多个文本进行索引嵌入。此方法的内部实现可能（但不一定）与嵌入查询的方式不同：

In [4]:
const text2 = "LangGraph is a library for building stateful, multi-actor applications with LLMs";

const vectors = await embeddings.embedDocuments([text, text2]);

console.log(vectors[0].slice(0, 100));
console.log(vectors[1].slice(0, 100));

[
      0.0190887451171875,  0.00482940673828125,   -0.0158233642578125,
       -0.04254150390625,    0.040130615234375,   -0.0268096923828125,
          -0.02392578125,    0.058624267578125,    0.0269927978515625,
          -0.04345703125,   0.0193328857421875,     0.040374755859375,
     -0.0196075439453125,  0.01384735107421875,    -0.021881103515625,
    -0.01068878173828125,   -0.016510009765625,   0.01079559326171875,
     -0.0227813720703125,        0.00634765625,     0.035064697265625,
     -0.0113983154296875,  -0.0257720947265625,   -0.0285491943359375,
       0.011749267578125,  -0.0222625732421875,   -0.0148468017578125,
        -0.0325927734375,  0.00203704833984375,      -0.0394287109375,
   -0.004878997802734375,  -0.0311126708984375,  -0.01248931884765625,
    -0.00897979736328125,  -0.0286407470703125,    0.0146331787109375,
        0.04522705078125,    0.050201416015625,    -0.053314208984375,
      0.0265960693359375,  -0.0207366943359375,      0.06658935546875,
    

## API 参考文档

如需详细了解所有 PineconeEmbeddings 功能和配置，请访问 API 参考文档：https://api.js.langchain.com/classes/_langchain_pinecone.PineconeEmbeddings.html